In [ ]:
#pip install tensorflow

In [1]:
pip install --upgrade tensorflow-hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade tensorflow-text

  Using cached tensorflow_text-2.10.0-cp39-cp39-win_amd64.whl (5.0 MB)
  Using cached tensorflow-2.10.1-cp39-cp39-win_amd64.whl (455.9 MB)
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.11.0
    Uninstalling tensorflow-2.11.0:
      Successfully uninstalled tensorflow-2.11.0



In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
prepro_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [ ]:
bert_preprocess_model = hub.KerasLayer(prepro_url)

In [ ]:
bert_model=hub.KerasLayer(encoder_url)

In [ ]:
sample_text=["hello world","you are the best"]

In [ ]:
sample_prepro=bert_preprocess_model(sample_text)

In [ ]:
sample_prepro.keys()

dict_keys(['input_mask', 'input_type_ids', 'input_word_ids'])

In [ ]:
print(sample_prepro['input_mask'])

tf.Tensor(
[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(2, 128), dtype=int32)


In [ ]:
sample_bert_result=bert_model(sample_prepro)

In [ ]:
sample_bert_result.keys()

dict_keys(['pooled_output', 'default', 'sequence_output', 'encoder_outputs'])

In [ ]:
print(sample_bert_result['encoder_outputs'])

[<tf.Tensor: shape=(2, 128, 768), dtype=float32, numpy=
array([[[ 0.19326207,  0.09833911, -0.01789953, ..., -0.04558702,
          0.00337627,  0.01849369],
        [ 0.66882664,  0.30082005,  0.5817627 , ..., -0.14621404,
          0.5790725 , -0.32852748],
        [ 0.3249809 ,  1.4820913 ,  0.626317  , ..., -0.46489853,
         -0.09663891, -0.34995088],
        ...,
        [-0.00404149, -0.14955175,  0.82427806, ...,  0.193481  ,
         -0.4235694 ,  0.07976594],
        [-0.09988259, -0.16565727,  0.6736971 , ...,  0.19536242,
         -0.3275261 , -0.00868384],
        [ 0.01640578, -0.08096911,  0.7214124 , ...,  0.51870656,
         -0.6287697 , -0.0495996 ]],

       [[ 0.21586266,  0.0739437 , -0.08171406, ...,  0.0645122 ,
          0.10520061,  0.07006498],
        [-0.5806993 ,  0.38480794,  0.9243454 , ...,  1.0549752 ,
          0.67878276,  0.08934157],
        [-0.24779887,  0.27915955,  0.91511333, ...,  0.08101156,
          0.4526126 ,  0.23193681],
        ...

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/smishing_data/data.csv')

In [ ]:
data.head()

,LABEL,TEXT
0,0,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,0,What's up? Do you want me to come online? If y...
2,0,So u workin overtime nigpun?
3,0,"Also sir, i sent you an email about how to log..."
4,1,Please Stay At Home. To encourage the notion o...


In [ ]:
data.groupby('LABEL').describe()

TEXT                                                               
      count unique                                                top freq
LABEL                                                                     
0      4846   4836  The greatest test of courage on earth is to be...    2
1       699    639  Had your mobile 11 months or more? U R entitle...    3
2       495    483  lyricalladie(21/F) is inviting you to be her f...    2

In [ ]:
X=data['TEXT']
Y=data['LABEL']

In [ ]:
temp=pd.DataFrame()

In [ ]:
X.shape

(6040,)

In [ ]:
tbatch_size = 512
start = 0
max_len = X.shape[0]
while start <= max_len:
  end = start+batch_size if start+batch_size < max_len else max_len
  print(f"Processing data from {start} to {end}")
  inputs=bert_preprocess_model(X[start:end])
  outputs=bert_model(inputs)
  for i in range(0,len(outputs['sequence_output'])):
    b=outputs['sequence_output'][i].numpy().sum(axis=0)
    temp=temp.append(pd.Series(b),ignore_index=True)
    file_name = f"/content/drive/MyDrive/bertFeatures{start}-{end}.csv"
    temp.to_csv( file_name, index=False, encoding='utf-8-sig')
  start = end
print('values added in dataframe')

Streaming output truncated to the last 5000 lines.
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 6040
Processing data from 6040 to 60

In [ ]:
outputs=bert_model(inputs)

In [ ]:
for i in range(0,len(outputs['sequence_output'])):
  b=outputs['sequence_output'][i].numpy().sum(axis=0)
  temp=temp.append(pd.Series(b),ignore_index=True)
  file_name = f"/content/drive/MyDrive/bertFeatures{start}-{end}.csv"
  temp.to_csv( file_name, index=False, encoding='utf-8-sig')
print('values added in dataframe')

values added in dataframe


In [ ]:
temp.head

<bound method NDFrame.head of            0          1          2          3          4          5    \
0   -13.751841 -44.407150  61.418320  -6.481122  25.783449 -31.427284   
1    28.586462 -18.739069  46.969658  23.832251  45.678162 -41.544392   
2    -3.579828 -19.711985  73.566681 -43.250164 -31.702173  23.378551   
3    10.045289 -38.011723  67.364182  32.802578  53.216183 -47.504307   
4    23.376081  -9.286799  67.754608  15.600671  26.961422 -24.874023   
..         ...        ...        ...        ...        ...        ...   
995  20.131512  -5.671147  28.622543  16.396336   3.309064  11.336006   
996  -7.298331  -5.628654  77.266167  -0.616204 -23.063290  -3.119037   
997  21.691607 -28.087801  54.922935   8.915645  27.046677   3.437698   
998  19.036570  -5.196006  41.875446  14.442719  28.912382  -2.862604   
999  11.766659 -41.642788  34.047508  36.700748  70.064507 -17.085894   

           6          7          8          9    ...        758        759  \
0     7.825738 

In [ ]:
temp.to_csv( "/content/drive/MyDrive/bertFeatures0-999.csv", index=False, encoding='utf-8-sig')

In [ ]:
print(outputs['sequence_output'])

tf.Tensor(
[[[-0.00374841 -0.44597927  0.42460805 ... -0.21206123  0.69609904
    0.5687282 ]
  [ 0.41150847 -0.39197755  0.2567888  ...  0.5725822   0.99874306
   -0.2190429 ]
  [ 0.08404027 -0.57623804 -0.02545114 ... -0.27296525 -0.32611364
    0.21915708]
  ...
  [ 0.22721413 -0.1915518   0.853736   ...  0.06003604  0.3099953
    0.4334102 ]
  [-0.01187555 -0.3502321   0.6774551  ...  0.04762588  0.0159026
    0.4364593 ]
  [ 0.16184527 -0.3805977   0.61297804 ...  0.09748667 -0.0703733
    0.43906474]]], shape=(1, 128, 768), dtype=float32)


In [ ]:
print(outputs['sequence_output'].shape)

(1, 128, 768)


In [ ]:
from sklearn.model_selection  import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size = 0.20, random_state=26)

In [ ]:
def berting(sentence):
  prepro=bert_preprocess_model(sentence)
  bert1=bert_model(prepro)
  return bert1['pooled_output']


In [ ]:
berting(["hello there"])

In [ ]:
import tensorflow as tf

In [ ]:
# Bert layers

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dense(2,activation='relu')(outputs['pooled_output'])
l = tf.keras.layers.Dropout(0.1, name="dropout")(l)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,batch_size=128,
    epochs=5,)